In [163]:
import pandas as pd
import numpy as np

In [164]:
sample_items_meli = pd.read_csv('sample_items_meli.csv')
# Caso queira limitar o número de amostras
# sample_items_meli = sample_items_meli.iloc[:1500]

In [165]:
categorias_distintas = len(set(sample_items_meli['ITE_ITEM_DOM_DOMAIN_ID']))
categorias_distintas

832

In [166]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import os

# Função para obter produtos similares do mesmo cluster
def obter_produtos_similares(df, item_id, top_n=5):
    # Obtém o cluster do item especificado
    cluster_id = df.loc[df['item_id'] == item_id, 'cluster'].values[0]
    # Seleciona todos os itens que estão no mesmo cluster
    itens_similares = df[df['cluster'] == cluster_id]
    # Exclui o próprio item da lista de similares
    itens_similares = itens_similares[itens_similares['item_id'] != item_id]
    # Retorna os top_n itens similares
    return itens_similares.head(top_n)


In [ ]:
# Função para carregar e achatar as imagens
def carregar_e_achatar_imagens(df, modo_cor='rgb', tamanho_imagem=(64, 64)):
    imagens = []
    ids = []
    for idx, row in df.iterrows():
        caminho_img = row['thumbnail_path']
        try:
            # Abre a imagem do caminho especificado
            img = Image.open(caminho_img)
            # Redimensiona a imagem para um tamanho padrão para uniformizar
            img = img.resize(tamanho_imagem)
            if modo_cor == 'grayscale':
                # Converte a imagem para escala de cinza
                img = img.convert('L')
                # Converte a imagem em um array numpy e achata em um vetor 1D
                img_array = np.array(img).flatten()
            else:
                # Garante que a imagem está em modo RGB
                img = img.convert('RGB')
                # Converte a imagem em um array numpy e achata em um vetor 1D
                img_array = np.array(img).flatten()
            # Adiciona o vetor de imagem à lista de imagens
            imagens.append(img_array)
            # Armazena o item_id correspondente
            ids.append(row['item_id'])
        except Exception as e:
            # Em caso de erro ao carregar a imagem, imprime o erro e continua
            print(f"Erro ao carregar a imagem {caminho_img}: {e}")
            continue
    # Retorna um array numpy das imagens e a lista de ids
    return np.array(imagens), ids


# === Recomendação usando imagens RGB ===
# Carrega e achata as imagens em RGB
imagens_rgb, ids_rgb = carregar_e_achatar_imagens(sample_items_meli, modo_cor='rgb', tamanho_imagem=(128, 128))
print(len(imagens_rgb))
print(len(ids_rgb))
print(imagens_rgb[0])

In [ ]:
# Função para realizar PCA e clustering nas imagens
def realizar_pca_clusterizacao(dados_imagem, n_componentes=50, n_clusters=10):
    # Normaliza os dados para ter média zero e variância unitária
    scaler = StandardScaler()
    dados_imagem_escalados = scaler.fit_transform(dados_imagem)
    
    # Aplica PCA para reduzir a dimensionalidade dos dados
    pca = PCA(n_components=n_componentes)
    dados_imagem_pca = pca.fit_transform(dados_imagem_escalados)
    
    # Aplica KMeans para agrupar os dados em clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(dados_imagem_pca)
    
    # Retorna os clusters e os dados transformados pelo PCA
    return clusters, dados_imagem_pca


# Realiza PCA e clustering nas imagens RGB
clusters_rgb, pca_rgb = realizar_pca_clusterizacao(
    imagens_rgb,
    n_componentes=50,
    n_clusters=categorias_distintas//2)
print(len(clusters_rgb))
print(len(pca_rgb))
print(clusters_rgb.shape)
print(pca_rgb.shape)
print(pca_rgb[0])
print(clusters_rgb)

In [ ]:
# Adiciona os clusters ao DataFrame original
sample_items_meli_rgb = sample_items_meli.copy()
sample_items_meli_rgb['cluster'] = pd.Series(clusters_rgb, index=sample_items_meli_rgb.index)
sample_items_meli_rgb

In [ ]:
# Exemplo: Obtém produtos similares para um item específico usando clusters RGB
item_id_exemplo = sample_items_meli_rgb['item_id'].iloc[42]
produtos_similares_rgb = obter_produtos_similares(sample_items_meli_rgb, item_id_exemplo)
print(f"Produtos similares (RGB) para o item {item_id_exemplo}:\n", produtos_similares_rgb)

In [ ]:
# === Recomendação usando imagens em escala de cinza ===
# Carrega e achata as imagens em escala de cinza
imagens_gray, ids_gray = carregar_e_achatar_imagens(sample_items_meli, modo_cor='grayscale', tamanho_imagem=(128, 128))

# Realiza PCA e clustering nas imagens em escala de cinza
clusters_gray, pca_gray = realizar_pca_clusterizacao(
    imagens_gray, 
    n_componentes=50, 
    n_clusters=categorias_distintas//2)

# Adiciona os clusters ao DataFrame original
sample_items_meli_gray = sample_items_meli.copy()
sample_items_meli_gray['cluster'] = pd.Series(clusters_gray, index=sample_items_meli_gray.index)

# Exemplo: Obtém produtos similares para um item específico usando clusters em escala de cinza
produtos_similares_gray = obter_produtos_similares(sample_items_meli_gray, item_id_exemplo)
print(f"Produtos similares (Escala de Cinza) para o item {item_id_exemplo}:\n", produtos_similares_gray)

In [ ]:
# === Recomendação usando SVD nas categorias compradas por cada usuário ===
# Cria uma matriz de compra de usuário por categoria
matriz_usuario_categoria = sample_items_meli.pivot_table(
    index='buyer_ID',
    columns='ITE_ITEM_DOM_DOMAIN_ID',
    aggfunc='size',
    fill_value=0
)

# Realiza a decomposição SVD
svd = TruncatedSVD(n_components=50, random_state=42)
fatores_usuario = svd.fit_transform(matriz_usuario_categoria)

# Função para recomendar categorias baseadas em usuários similares
def recomendar_categorias(usuario_id, fatores_usuario, matriz_usuario_categoria, top_n=5):
    # Obtém o índice do usuário na matriz
    idx_usuario = matriz_usuario_categoria.index.get_loc(usuario_id)
    # Obtém o vetor de fatores do usuário
    vetor_usuario = fatores_usuario[idx_usuario].reshape(1, -1)
    # Calcula a similaridade coseno entre o usuário e todos os outros usuários
    similaridades = cosine_similarity(vetor_usuario, fatores_usuario)[0]
    # Ordena os usuários pela similaridade (excluindo o próprio usuário)
    idx_usuarios_similares = np.argsort(-similaridades)[1:]
    usuarios_similares = matriz_usuario_categoria.index[idx_usuarios_similares]
    # Soma as categorias compradas pelos usuários similares
    categorias = matriz_usuario_categoria.loc[usuarios_similares].sum()
    # Remove as categorias que o usuário já comprou
    categorias_usuario = matriz_usuario_categoria.loc[usuario_id]
    categorias_recomendar = categorias[categorias_usuario == 0]
    # Recomenda as top_n categorias
    categorias_recomendadas = categorias_recomendar.sort_values(ascending=False).head(top_n).index.tolist()
    return categorias_recomendadas

# Exemplo: Recomenda categorias para um usuário específico
usuario_id_exemplo = matriz_usuario_categoria.index[0]
categorias_recomendadas = recomendar_categorias(usuario_id_exemplo, fatores_usuario, matriz_usuario_categoria)
print(f"Categorias recomendadas para o usuário {usuario_id_exemplo}:\n", categorias_recomendadas)
